In [ ]:
import requests
import pandas as pd
from datetime import datetime
import time
import json

def fetch_weibo_hotsearch_api():
    """
    通过微博API接口获取热搜数据
    返回DataFrame格式的热搜数据
    """
    url = 'https://weibo.com/ajax/side/hotSearch'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            print(f"请求失败，状态码：{response.status_code}")
            return None

        return response.json()

    except Exception as e:
        print(f"获取数据时出错：{e}")
        return None

def hotseach_info_process()
        # 提取热搜数据
        hot_list = []
        # 处理置顶热搜
        if 'hotgov' in data and data['hotgov']:
            hotgov = data['hotgov']
            hot_list.append({
                'rank': 0,
                'title': hotgov['word'].strip('#'),
                'url': f"https://s.weibo.com/weibo?q={hotgov['word']}",
                'hot_value': hotgov.get('raw_hot', 0),
                'label': '置顶',
                'is_ad': False
            })

        # 处理普通热搜
        if 'realtime' in data:
            for i, item in enumerate(data['realtime']):
                hot_list.append({
                    'rank': i + 1,
                    'title': item['word'],
                    'url': f"https://s.weibo.com/weibo?q={item['word']}",
                    'hot_value': item.get('raw_hot', 0),
                    'label': item.get('label_name', ''),
                    'is_ad': 'is_ad' in item and item['is_ad'] == 1
                })

        # 转换为DataFrame
        df = pd.DataFrame(hot_list)

        # 添加时间戳
        df['crawl_time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        return df



# 使用示例
hotsearch_df = fetch_weibo_hotsearch_api()

if hotsearch_df is not None:
    print(f"成功获取{len(hotsearch_df)}条热搜数据")
    print(hotsearch_df.head(10))

    # 保存到CSV文件
    filename = f"weibo_hotsearch_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    hotsearch_df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"数据已保存到：{filename}")
else:
    print("获取热搜数据失败")